# WORKFLOW 1.1 : LLM Agents + Tool Use (Yelp API)

## 1.1.1 Tool for the agent

In [2]:
# Import the modules
import requests
import json
import os

In [12]:
# Load env
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [4]:
# Define my API Key, My Endpoint, and My Header
YELP_API_KEY = os.environ['YELP_API_KEY']
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {'Authorization': 'bearer %s' % YELP_API_KEY}

In [6]:
from typing_extensions import Annotated

def get_places(
    term: Annotated[str, "A keyword or search term to use to search for places with this term"], 
    location: Annotated[str, "a string which is the user's current location"], 
    radius: Annotated[int, "the radius in metres to provide the places result for"]
    ) -> Annotated[str, "Results in json format"]:
    
    # Define my parameters of the search
    # BUSINESS SEARCH PARAMETERS - EXAMPLE
    PARAMETERS = {'term': term,
                 'limit': 10,
                 'offset': 10,
                 'radius': radius,
                 'location': location,
    #              'latitude': 41.836609,
    #              'longitude': -87.616276,
    #              'sort_by':'distance',
                 }
    # Make a request to the Yelp API
    response = requests.get(url = ENDPOINT,
                        params = PARAMETERS,
                        headers = HEADERS)
    return response.json()

## 1.1.2 Building the Agentic Workflow

### Creating Agents

In [14]:
from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
travel_agent = ConversableAgent(
    name="Agent",
    system_message="You are a helpful AI food & travel assistant. "
    "You can help with getting relevant places based on the users query. You will suggest a tool to complete the query."
    "You will ask the required data or arguements required for the function if the user has not already given it."
    "You will give the final answer to the user in the form of a list, where each entry is of the format 'name', 'distance'"
    "Do not display additional information until asked by the user"
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="ALWAYS",
)

### Register Tool

In [9]:
from autogen import register_function

# Register the function to the two agents.
register_function(
    get_places,
    caller=travel_agent,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="get_places",    # By default, the function name is used as the tool name.
    description="""
                    A function used to return the places in a json formatted file, based on the user's query. 
                    The arguements are,
                    term - the keyword or search term to use to search for places with this term;
                    location - a string which is the user's current location;
                    radius - the radius in metres to provide the places result for;
                """,  # A description of the tool.
)

### Run the Workflow

In [10]:
chat_result = user_proxy.initiate_chat(travel_agent, message="Give me burger places within 1200 metres")

User (to Agent):

Give me burger places within 1200 metres

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Agent (to User):

Please provide me with your current location so I can find burger places within 1200 metres.

--------------------------------------------------------------------------------
Provide feedback to Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 401 East, 32nd Street, Chicago
User (to Agent):

401 East, 32nd Street, Chicago

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Agent (to User):

***** Suggested tool call (call_qVBHW0dofCnHswJVSVDHCVGv): get_places *****
Arguments: 
{"term":"burger","location":"401 East, 32nd Street, Chicago","radius":1200}
***************************************************************************

------------------------------------------------------------------------------

**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Agent (to User):

Here are some burger places within 1200 metres of your location:

1. 'Wood's Food and Liquor', '730.98 m'
2. '7-Eleven', '843.53 m'
3. 'White Castle', '548.79 m'
4. 'Best Southtown Gyro', '668.12 m'
5. 'Morrison's Soul Food', '1353.16 m'
6. 'McDonald's', '1184.91 m'
7. 'McDonald's', '749.80 m'

TERMINATE

--------------------------------------------------------------------------------
Provide feedback to Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Give me the address of the nearest McD
User (to Agent):

Give me the address of the nearest McD

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Agent (to User):

The nearest McDonald's is located at:

'207 E 35th St, Chicago, IL 60653'

If 

## References:

https://microsoft.github.io/autogen/docs/tutorial/tool-use \
https://github.com/areed1192/sigma_coding_youtube/blob/master/python/python-api/yelp-api/Yelp%20API%20-%20Business%20Search.py